<a href="https://colab.research.google.com/github/Keremalikorkmaz/Create-Teach-Evaluate/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dumbelek
from dumbelek import Cleaner
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import pandas as pd
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import warnings
import numpy as np
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
warnings.filterwarnings("ignore")

In [ ]:
#Oluştur

Media=["Media", "News", "Televison", "radio", "publisihing"]
Art = ["Art", "Artist", "Art", "Artistic"]
Tourism = ["Tourism", "Tourist", "Touristic", "Travel", "Vacation"]
categories = [Tourism,Art,Media]



df=pd.DataFrame()
for j in range(len(categories)):
    text = ''
    for i in range(len(categories[j])):
        r = requests.get(f"https://sentence.yourdictionary.com/{categories[j][i]}")
        soup = bs(r.content)
        paragraph =soup.find_all("span", attrs={"class": "sentence-item__text"})
        text = text + str(paragraph)
    arr = text.split("</span>,")
    arr=[s.replace(' <span class="sentence-item__text" data-v-833c1076="">','') for s in arr]
    arr=[s.replace('<mark>','') for s in arr]
    arr=[s.replace('</mark>','') for s in arr]
    arr=[s.replace('[<span class="sentence-item__text" data-v-833c1076="">','') for s in arr]
    arr=[s.replace('</span>]','') for s in arr]
    df2 = pd.DataFrame(arr,columns=["Sentences"])
    df2["Category"]=categories[j][0]
    df=pd.concat([df,df2])
    
df['CategoryID'] = labelencoder.fit_transform(df['Category'])    
df.reset_index(drop=True, inplace=True)    
   


In [ ]:
df.to_excel("Makale4.xlsx",index=False) 
df.head()

,Sentences,Category,CategoryID
0,VisitNC.com - See the official tourism website...,Tourism,2
1,It is their aim to ensure that aboriginal tour...,Tourism,2
2,"The off-season for tourism, late fall through ...",Tourism,2
3,I reckon tourism is the most popular attractio...,Tourism,2
4,The new leaflet looks at some examples of tour...,Tourism,2


In [ ]:
def clean(text):
  text = Cleaner.clean_text(text)
  return text

In [ ]:
df["Sentences"] = df["Sentences"].apply(clean)

In [ ]:
df.head()

,Sentences,Category,CategoryID
0,visitnc com see the official tourism website f...,Tourism,2
1,it is their aim to ensure that aboriginal tour...,Tourism,2
2,the off season for tourism late fall through w...,Tourism,2
3,increased affluence from tourism has also resu...,Tourism,2
4,while the initial costs for tourism will be hi...,Tourism,2


In [ ]:
df['Category'].value_counts()

Art        1542
Tourism    1288
Media      1075
Name: Category, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.CategoryID.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.CategoryID.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
label_dict  = {'ARTS': 0, 'BUSINESS & FINANCE': 1, 'EDUCATION': 2, 'FOOD': 3, 'HEALTH': 4, 
               'MEDIA': 5, 'NATURE & GEOGRAPHY': 6, 'POLITICS': 7, 'RELIGION': 8, 'SCIENCE & TECH': 9,
               'SPORTS': 10, 'HISTORY': 11, 'LAW & HUMAN RIGHTS': 12, 'MILITARY': 13, 'TOURISM': 14}